# Dengarkan Analisis Serangan dan Kumpulan Data Prediksi

Dataset ini berisi informasi tentang orang-orang dan ada


 **Dataset Information:**





* Umur: Usia pasien
* Jenis Kelamin: Jenis Kelamin pasien
* exang: angina akibat olahraga (1 = ya; 0 = tidak)
* ca: jumlah kapal besar (0-3)
* cp: Jenis nyeri dada jenis nyeri dada
    * Value 1: typical angina
    * Value 2: atypical angina
    * Value 3: non-anginal pain
    * Value 4: asymptomatic
* trtbps: tekanan darah (dalam mm Hg)
* chol: kolestoral dalam mg / dl diambil melalui sensor BMI
* fbs: (gula darah puasa> 120 mg / dl) (1 = true; 0 = false)
* rest_ecg: hasil elektrokardiografik istirahat
    * Value 0: normal
    * Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    * Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
* thalach: detak jantung maksimum tercapai
* target : 
    * 0 = less chance of heart attack 
    * 1 = more chance of heart attack
    
    

**Objective:**




* Dengan dataset yang disediakan untuk analisis jantung, kita harus menganalisis kemungkinan serangan jantung berdasarkan berbagai fitur, dan kemudian prediksi dari analisis tersebut akan memberi tahu kita apakah seseorang rentan terhadap serangan jantung atau tidak. 
* Analisis rinci dapat dilanjutkan dengan analisis data eksplorasi (EDA). 
* Klasifikasi untuk predikasi dapat dilakukan dengan menggunakan berbagai algoritma model pembelajaran mesin, memilih model yang paling sesuai untuk analisis serangan jantung dan terakhir menyimpan model dalam file(.pkl).


**Questions to be answered:**





* Apakah usia seseorang berkontribusi terhadap serangan jantung?
* Apakah jenis nyeri dada yang berbeda terkait satu sama lain atau kemungkinan terkena serangan jantung?
* Apakah tekanan darah tinggi meningkatkan risiko serangan jantung?
* Apakah kadar choestrol pada akhirnya berkontribusi sebagai faktor risiko terhadap serangan jantung?



In [ ]:
import numpy as np #linear algebra
import pandas as pd #data processing

import matplotlib.pyplot as plt #data visualization
import seaborn as sns #data visualization

import warnings
warnings.filterwarnings("ignore") #to ignore the warnings

#for model building
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb


In [ ]:
#Membaca file csv heart.csv dalam variabel 
df=pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')

In [ ]:
# melihat 5 baris pertama dari data kami
df.head()

**Observation:**
 
 * Sepertinya semua kolom sudah dalam tipe data int atau float.
 * Jika kolom tidak dalam int atau float (yaitu variabel kategori), kita harus mengubahnya untuk pembuatan model.
 * Beberapa cara untuk melakukannya adalah dengan menggunakan pd.get_dummies, satu hot encoding, multi collinearity, label encoder, dll.

In [ ]:
df.tail()

In [ ]:
print('Number of rows are',df.shape[0], 'and number of columns are ',df.shape[1])

In [ ]:
df.info()

**Observation:**

* Tidak ada baris yang hilang di seluruh kumpulan data.
* Semua kolom kecuali oldpeak (float) adalah tipe data int.

In [ ]:
df.isnull().sum()

**Observation:** Tidak ada nilai yang hilang.

In [ ]:
df.columns

In [ ]:
df.duplicated().sum()

**Observation:** Ada 1 baris duplikat. Ayo jatuhkan!

In [ ]:
df.drop_duplicates(inplace=True)
print('Number of rows are',df.shape[0], 'and number of columns are ',df.shape[1])

In [ ]:
df.describe().T

**Observation:**

* Tekanan darah rata-rata seseorang adalah 130 sedangkan nilai maksimun naik menjadi 200.
* Denyut jantung rata-rata pada grup adalah 152, sedangkan secara keseluruhan berkisar antara 133 hingga 202
* Usia kelompok bervariasi dari 29 hingga 77 dan usia rata-rata adalah 55,5

In [ ]:
#Ini untuk melihat apa yang dimiliki semua nilai unik. Hanya mencoba menggunakan python
list_col=['sex','chol','trtbps','cp','thall','exng']

for col in list_col: 
    print('{} :{} ' . format(col.upper(),df[col].unique()))

**Observation:**

* Ada dua jenis kelamin: 0 dan 1
* Kadar kolestrol tertinggi adalah 564 dan terendah 126.
* Tekanan Darah Istirahat individu bervariasi antara 94 hingga 200.
* Ada 4 jenis nyeri dada.
* angina yang diinduksi latihan memiliki 2 jenis (1 = ya; 0 = tidak)

In [ ]:
print(f'Number of people having sex as 0 are {df.sex.value_counts()[0]} and Number of people having sex as 1 are {df.sex.value_counts()[1]}')
p = sns.countplot(data=df, x="sex", palette='pastel')
plt.show()

**Observation:** 

* Jumlah orang yang termasuk dalam kategori seks 0 adalah 96 sedangkan 1 adalah 206.
* Jumlah orang dalam satu kategori lebih dari dua kali lipat dari nol.

In [ ]:
sns.countplot(x='cp', data=df, palette='pastel')

**Observation:**

* cp: Jenis nyeri dada jenis nyeri dada

     * Nilai 0: angina khas
     * Nilai 1: angina atipikal
     * Nilai 2: nyeri non anginal
     * Nilai 3: asimtomatik
    
* Orang dengan kategori nyeri dada '0' memiliki jumlah tertinggi, sedangkan jumlah nyeri dada '3' adalah yang terendah

In [ ]:
sns.countplot(x='fbs', data=df, palette='pastel')

**Observation:** Orang fbs kategori 1 kurang dari 25% orang fbs kategori 0.

In [ ]:
sns.countplot(x='thall', data=df, palette='pastel')

**Observation:** Jumlah total maksimal untuk tipe 2 (165) dan min untuk tipe 0 (2).


In [ ]:
sns.countplot(x='restecg', data=df, palette='pastel')

**Observation:** 

* Hitungan EKG hampir sama untuk tipe 0 dan 1.
* Juga, hampir dapat diabaikan untuk tipe 2 dibandingkan dengan tipe 0 dan 1.

In [ ]:
plt.figure(figsize = (10,10))
sns.swarmplot(x=df['caa'],y=df['age'],hue=df['output'], palette='pastel')

**Observation:**

* Petak-petak gerombolan ini memberi kita banyak informasi.
* Berdasarkan gambar, orang yang termasuk kategori caa '0', berapa pun usianya, sangat rentan terkena serangan jantung.
* Meskipun sangat sedikit orang yang termasuk dalam kategori caa '4', tetapi tampaknya sekitar 75% dari mereka mengalami serangan jantung.
* Orang-orang yang termasuk dalam kategori '1', '2' dan '3' kurang lebih memiliki risiko yang sama.

In [ ]:
sns.color_palette("pastel")
plt.title('Checking Outliers with distplot()')
sns.distplot(df.trtbps, label='trtbps', kde=True, bins=10, color='green')
plt.legend()

In [ ]:
plt.title('Checking Outliers with distplot()')
sns.distplot(df.chol, label='chol', kde=True, color='red')
plt.legend()

In [ ]:
plt.title('Checking Outliers with distplot()')
sns.distplot(df['thalachh'],label='thalachh', kde=True )
plt.legend()

**Observations:**

* trtbps dan chol sepertinya terdistribusi normal, dengan beberapa pencilan sangat condong ke kanan.
* Dalam kasus thalachh, datanya sangat condong ke kanan!

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x = df['age'], y = df['thall'],marker = '*', linestyle = '--', color = 'red')

plt.figure(figsize = (20,10))
sns.regplot(x=df['age'],y=df['oldpeak'])

**Observation:** 

* Ini hampir tidak memberikan informasi apa pun.
* Hubungan antara age-oldpeak dan age-thall sangat tidak pasti dan sangat bervariasi.

In [ ]:
X = df.drop('output', axis = 1)
y = df['output']

**About StandardScaler:**

* Library sklearn Python menawarkan kepada kita fungsi StandardScaler () untuk menstandarkan nilai data ke dalam format standar.

* Standarisasi kumpulan data adalah persyaratan umum untuk banyak penaksir pembelajaran mesin: mereka mungkin berperilaku buruk jika fitur individu tidak lebih atau kurang terlihat seperti data terdistribusi normal standar (misalnya Gaussian dengan 0 mean dan unit varians).

* StandardScaler mengikuti Standard Normal Distribution (SND). Oleh karena itu, membuat mean = 0 dan menskalakan data ke unit varians.

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
columns_to_scale = df.iloc[:,[0,3,4,7,9,]]
columns_to_scale

In [ ]:
# Membagi data
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

ss = StandardScaler()
scaled_values = ss.fit_transform(columns_to_scale)
scaled_values = pd.DataFrame(scaled_values, columns=columns_to_scale.columns)
scaled_values


In [ ]:
scaled_df = pd.concat([scaled_values,df.iloc[:,[1,2,5,6,8,10,11,12,13]]],axis=1)
scaled_df

In [ ]:
key = ['LogisticRegression','KNeighborsClassifier','SVC','DecisionTreeClassifier','RandomForestClassifier','GradientBoostingClassifier','AdaBoostClassifier','XGBClassifier']
value = [LogisticRegression(random_state=9), KNeighborsClassifier(), SVC(), DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(), AdaBoostClassifier(), xgb.XGBClassifier()]
models = dict(zip(key,value))

In [ ]:
predicted =[]

In [ ]:
for name,algo in models.items():
    model=algo
    model.fit(X_train,y_train)
    predict = model.predict(X_test)
    acc = accuracy_score(y_test, predict)
    predicted.append(acc)
    print(name,acc)

In [ ]:
plt.figure(figsize = (10,5))
sns.barplot(x = predicted, y = key, palette='pastel')

**Observation:**  Dari gambar di atas kita dapat melihat bahwa tidak ada model di atas yang memberikan akurasi lebih dari 90%. Mari kita coba pendekatan lain. Mari kita gunakan random_state lain untuk Model Regresi Logistik dan lihat apakah akurasinya meningkat!

In [ ]:
lr = LogisticRegression(solver='lbfgs', max_iter=10000)
rs = []
acc = []
for i in range(1,100,1):
    X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.2, random_state = i)    
    model_lr_rs = lr.fit(X_train, y_train.values.ravel())
    predict_values_lr_rs = model_lr_rs.predict(X_test)
    acc.append(accuracy_score(y_test, predict_values_lr_rs))
    rs.append(i)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(rs, acc)

In [ ]:
for i in range(len(rs)):
    print(rs[i],acc[i])

**Observation:** Jadi akurasi prediksi terbaik untuk peluang Serangan Jantung adalah 91,8. Tetapi dengan keadaan acak yang berbeda, kita harus lebih mengandalkan rata-rata dari semua akurasi.

# Kesimpulan: 


* Variabel Numerik - Tidak ada pencilan yang ditemukan!

* Tekanan Darah Tinggi, Kolestrol Tinggi dan Denyut Jantung Tinggi menyebabkan kemungkinan tinggi serangan jantung.

* Dalam hitungan target muncul bahwa kita memiliki lebih banyak rincian serangan jantung.

* Usia 40-60 tahun memiliki kemungkinan tinggi terkena serangan jantung.

* Jenis kelamin laki-laki lebih besar kemungkinannya terkena serangan jantung dibandingkan dengan perempuan.

* Faktor yang Sangat Berkorelasi dalam kumpulan data ini adalah:
     * Usia dan trtbps (laju tekanan darah)
     * Umur dan kol (tingkat kolestrol)